In [ ]:
import base64
import json
import requests
from os import getenv
from dotenv import load_dotenv

load_dotenv()

In [ ]:
public_key = getenv("PUBLIC_KEY")
private_key = getenv("PRIVATE_KEY")
dev_url = getenv("DEV_URL")
prod_url = getenv("PROD_URL")
header = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}
public_header = header | {
    "Authorization": public_key,
}
private_header =  header | {
    "Authorization": private_key,
}

In [ ]:
company_resp = requests.get(f"{dev_url}/company", headers=public_header)

In [ ]:
company_resp.text

In [ ]:
with open("big_data_chunks/contrato_arrendamiento.pdf", "rb") as documento:
    encoded_string = base64.b64encode(documento.read()).decode('UTF-8')

In [ ]:
payload = {
    "name": "Contrato de Arrendamiento Dora!",
    "message": "Hola {{ nombre_firmante }}, les comparto el siguiente pdf para que lo firmen",
    "subject": "Contrato de arrendamiento entre {{ arrendador }} y {{ arrendatario }}",
    "email": "hola@holadora.co",
    "remember": 6,
    "signProfile": [
        {
            "type": "arrendatario",
            "name": "Juan Francisco Garcia",
            "email": "sebaslander@gmail.com",
            "phone": "3137657150",
            "position": [
               {
                   "page": 7,
                   "x": 0.126,
                   "y": 0.62,
                   "w": 100,
                   "h": 50
               }
            ]
        },
        {
            "type": "arrendador",
            "name": "Grupo Relead",
            "email": "sebaslander@yahoo.es",
            "phone": "3137657150",
            "position": [
               {
                   "page": 7,
                   "x": 0.126,
                   "y": 0.86,
                   "w": 100,
                   "h": 50
               }
            ]
        },
    ],
    "camera": True,
    "otpCode": True,
    "options": {
        "camera": "identification",
        "otpCode": "phone",
        "whatsapp": True
    },
    "file": encoded_string
}

In [ ]:
upload_doc_resp = requests.post(f"{dev_url}/document/upload", headers=private_header, json=payload) # json=json.dumps(payload)

In [ ]:
upload_doc_resp.json()

In [ ]:
doc = requests.get(f"{dev_url}/document?code={upload_doc_resp.json().get('document', None)}", headers=public_header)

In [ ]:
doc.content

In [ ]:
## URLs para revisar implementación de NGrok
"""
https://ngrok.com/
https://adamj.eu/tech/2021/05/09/how-to-build-a-webhook-receiver-in-django/
https://swapps.com/blog/how-to-add-webhooks-to-a-django-project-and-test-them-locally/
https://tech.osteel.me/posts/docker-for-local-web-development-part-6-expose-a-local-container-to-the-internet
"""

In [ ]:
def _crear_url(endpoint: str):
    """Metodo auxiliar para crear la URL del endpoint de manera dinamica.

    Args:
        endpoint (str): URL del enpoint, se requiere que incluya el caracter '/'.

    Returns:
        str: La URL construida dinamicamente, usando la base de la instancia y el endpoint a llamar.
    """
    return f"{dev_url}{endpoint}"

def _establecer_conexion(
        endpoint: str,
        privado: bool = False,
        accion: str = "get",
        payload: dict = None,
    ):
    """Metodo de clase del modelo, que establece la conexión con el

    Args:
        endpoint (str): cadena final del endpoint
        privado (bool, optional): variable usada para definir los headers, que cambian de acuerdo
                                a lo que requiera el endpoint invocado. Defaults to False.
        accion (str, optional): verbo de llamada requerido en el endpoint. Defaults to "get".
        payload (dict, optional): diccionario. Defaults to None.

    Returns:
        Dict[str, Any]: Objeto tipo String con el token de autenticación requerido para las consultas.
    """
    shared_head = {"Accept": "application/json", "Content-Type": "application/json"}
    public_headers = shared_head | {
        "Authorization": public_key,
    }
    private_headers = shared_head | {
        "Authorization": private_key,
    }
    headers = private_headers if privado else public_headers
    try:
        if accion == "get":
            response = requests.get(_crear_url(endpoint), headers=headers)
            response.raise_for_status()

        elif accion == "post":
            response = requests.post(
                _crear_url(endpoint), headers=headers, json=payload
            )
            response.raise_for_status()
    except Exception as e:
        print("Upppsss", e)
    
    return response.json()

In [ ]:
payload_company = {
            "webhook": getenv("URL_NGROK"),
            "webhookHeader": {
                "Authorization": getenv("TOKEN_TEMP_AUCO"),
            },
        }
another_payload = {
            "webhook": "https://c430-181-62-52-115.ngrok.io/api/documento"
        }
and_other_one = {
            "webhook": "https://c430-181-62-52-115.ngrok.io/api/documento",
            "webhookHeader": "NJxkpL9dP!naYnb?zigFc&C@a#tXY5iNy9rNq8Nc",
        }

In [ ]:
_establecer_conexion("document/upload", privado=True, accion="post", payload=payload)

In [ ]:
response = requests.put("https://dev.auco.ai/v1/ext/company", headers=private_header, json=and_other_one)

In [ ]:
response.json()

In [ ]:
payload = {
    "codes": "GXN8MQLW75",
    "message": "Cancelación manual por el usuario"
}

def cancelar_firmar_documento():
        payload = {
            "codes": ["aksdjhfalkjdhf",],
            "message": "Cancelación manual por el usuario",
        }
        respuesta = _establecer_conexion(
            "document/cancel", privado=True, accion="post", payload=payload
        )
        return respuesta

In [ ]:
cancelar_firmar_documento()